# SI Opportunity Scoring — Stakeholder Notebook (v6)
**Three methods, one validation: Fixed rule → Weighted (data-driven) rule → ML (Calibrated Logistic Regression)**  
**Order:** Cleaning/filtering → Label/ID aggregation → Encoding/Features → Train/Validation → Evaluate 3 methods → Operational output  
**Last updated:** 2026-02-27

## Goal
Identify **IDs with `si_offering = 0`** (not currently in an SI offering) that are **most likely to be interested in SI**.

## Proxy target (for benchmarking only)
We **derive** `si_offering` from `OFFERING_NAME`:
- `si_offering_row = 1` if OFFERING_NAME contains token **SI** (case-insensitive)
- `si_offering` (per ID) = max across that ID

> This is a *proxy* label: membership ≠ true interest.  
> The pilot section explains how to create true labels (response/adoption) and retrain.

## Business logic (requested)
- **If `MIFID = 0`:** score is based **only** on `SI_CONSIDERATION`  
  - S1: **no** interest  
  - S2: **some** interest  
  - S3: **high** interest  
- **If `MIFID = 1`:** we only use **SFDR / PAI / Taxonomy** signals when **`SI_CONSIDERATION` is high (S3)**  
  - If SI is not high, keep a low/medium baseline and do not “invent” preferences.

This logic is implemented in **Fixed rule** and **Weighted rule**.  
ML is used as a controlled enhancement, but we still encode the same structure via interaction features.

---
## 0) Setup

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 200)

---
## 1) Load raw data + shuffle rows

We shuffle to avoid any ordering artifacts in later analysis.

In [ ]:
DATA_PATH = Path("data.csv")  # <-- change to your real file path

def make_synthetic_data(n=9000, seed=42):
    rng = np.random.default_rng(seed)
    return pd.DataFrame({
        "ID": rng.integers(1, n//2 + 1, size=n),
        "IO_TYPE": rng.choice(["normal", "zombie"], size=n, p=[0.97, 0.03]),
        "LIFE_CYCLE": rng.choice(["open", "closed"], size=n, p=[0.9, 0.1]),
        "OFFERING_NAME": rng.choice(
            ["Core", "Standard", "ESG Plus", "SI Focus", "Core SI", "Income", "SI Sustainable", None],
            size=n, p=[0.23,0.23,0.14,0.14,0.08,0.07,0.06,0.05]
        ),
        "SI_CONSIDERATION_CD": rng.choice(["S1","S2","S3", None], size=n, p=[0.35,0.35,0.2,0.1]),
        "SFDR_PREF": rng.choice(["F1","F2","F3", None], size=n, p=[0.4,0.35,0.2,0.05]),
        "SFDR_ACTUAL": rng.choice(["F1","F2","F3", None], size=n, p=[0.45,0.35,0.15,0.05]),
        "PAI_PREF": rng.choice(["PAI Selected", None], size=n, p=[0.3,0.7]),
        "MIFID": rng.choice(["Yes","No", None], size=n, p=[0.55,0.4,0.05]),
        "TAXONOMYPREF": rng.choice(["A1","A2","A3", None], size=n, p=[0.5,0.35,0.1,0.05]),
        "GHG": rng.choice(["Yes","No","--", None], size=n, p=[0.25,0.65,0.05,0.05]),
        "Biodiversity": rng.choice(["Yes","No","--", None], size=n, p=[0.2,0.7,0.05,0.05]),
        "Water": rng.choice(["Yes","No","--", None], size=n, p=[0.22,0.68,0.05,0.05]),
        "Waste": rng.choice(["Yes","No","--", None], size=n, p=[0.18,0.72,0.05,0.05]),
        "Social": rng.choice(["Yes","No","--", None], size=n, p=[0.28,0.62,0.05,0.05]),
    })

if DATA_PATH.exists():
    df_raw = pd.read_csv(DATA_PATH)
    print(f"Loaded: {DATA_PATH}  shape={df_raw.shape}")
else:
    df_raw = make_synthetic_data()
    print("DATA_PATH not found; using synthetic demo dataset.")
    print(f"shape={df_raw.shape}")

df_raw = df_raw.sample(frac=1, random_state=42).reset_index(drop=True)
df_raw.head()

---
## 2) Cleaning & filtering (ONLY)

This step is **data hygiene only**.

**We do:** filter out non-actionable rows and standardize missing placeholders.
**We do NOT:** map categories to numbers here.

In [ ]:
REQUIRED_RAW = [
    "ID","IO_TYPE","LIFE_CYCLE","OFFERING_NAME",
    "SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","PAI_PREF","MIFID","TAXONOMYPREF",
    "GHG","Biodiversity","Water","Waste","Social"
]
missing_cols = [c for c in REQUIRED_RAW if c not in df_raw.columns]
if missing_cols:
    raise ValueError(f"Missing required raw columns: {missing_cols}")

def clean_filter_only(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # strip whitespace for object cols
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # standardize placeholder missing
    df = df.replace({"--": np.nan, "": np.nan})

    before = len(df)
    df = df[df["IO_TYPE"].fillna("").str.lower() != "zombie"]
    after_zombie = len(df)
    df = df[df["LIFE_CYCLE"].fillna("").str.lower() == "open"]
    after_open = len(df)

    df.attrs["cleaning_summary"] = {
        "before": before,
        "after_remove_zombie": after_zombie,
        "after_keep_open": after_open,
        "removed_zombie": before - after_zombie,
        "removed_closed": after_zombie - after_open
    }
    return df

df_clean_rows = clean_filter_only(df_raw)
pd.DataFrame([df_clean_rows.attrs["cleaning_summary"]])

In [ ]:
# Charts: cleaning impact + missingness snapshot
s = df_clean_rows.attrs["cleaning_summary"]
plt.figure(figsize=(8,4))
plt.bar(["Before", "After zombie", "After open"], [s["before"], s["after_remove_zombie"], s["after_keep_open"]])
plt.title("Cleaning impact: rows remaining after filters")
plt.ylabel("Rows")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

key_cols = ["SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","TAXONOMYPREF","MIFID","PAI_PREF",
            "GHG","Biodiversity","Water","Waste","Social","OFFERING_NAME"]
miss = df_clean_rows[key_cols].isna().mean().sort_values(ascending=False)
display(miss.to_frame("missing_rate").head(12))

plt.figure(figsize=(10,4))
plt.bar(miss.index[:12], miss.values[:12])
plt.title("Top missing rates after cleaning (raw values, before encoding)")
plt.ylabel("Missing rate")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 3) Derive proxy label and aggregate to ID-level

We now derive `si_offering` and aggregate to one row per ID (unit of action).

In [ ]:
df = df_clean_rows.copy()

# Row-level SI membership derived from OFFERING_NAME
df["si_offering_row"] = df["OFFERING_NAME"].astype(str).str.contains(r"\bSI\b", case=False, na=False).astype(int)

def mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if len(s2) == 0:
        return np.nan
    m = s2.mode()
    return m.iloc[0] if len(m) else s2.iloc[0]

agg_dict = {
    "IO_TYPE": mode_or_first,
    "LIFE_CYCLE": mode_or_first,
    "OFFERING_NAME": mode_or_first,  # transparency only (not used as feature)
    "SI_CONSIDERATION_CD": mode_or_first,
    "SFDR_PREF": mode_or_first,
    "SFDR_ACTUAL": mode_or_first,
    "PAI_PREF": mode_or_first,
    "MIFID": mode_or_first,
    "TAXONOMYPREF": mode_or_first,
    "GHG": mode_or_first,
    "Biodiversity": mode_or_first,
    "Water": mode_or_first,
    "Waste": mode_or_first,
    "Social": mode_or_first,
    "si_offering_row": "max",
}

df_id = df.groupby("ID", as_index=False).agg(agg_dict).rename(columns={"si_offering_row":"si_offering"})

sizes = pd.DataFrame({
    "level": ["row-level (cleaned)", "ID-level"],
    "rows": [len(df), len(df_id)],
    "si_offering_rate": [df["si_offering_row"].mean(), df_id["si_offering"].mean()]
})
display(sizes)

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["rows"])
plt.title("Size: cleaned rows vs ID-level")
plt.ylabel("Count")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["si_offering_rate"])
plt.title("Proxy label prevalence: si_offering rate")
plt.ylabel("Rate")
plt.ylim(0, 1)
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

df_id.head()

---
## 4) Feature encoding & engineering (numbers happen here)

This section converts business fields into numeric signals for scoring and ML.

### Key engineered signals
- `sfdr_gap = clip(pref - actual, -2, 2)`; `sfdr_opp = max(sfdr_gap, 0)`
- `pai_block` = 0 if no PAI, else 0.5 + 0.5*topics_norm
- `tax_norm` scales A1/A2/A3 → 0..1

In [ ]:
MAP_SI = {"S1":1, "S2":2, "S3":3}
MAP_SFDR = {"F1":1, "F2":2, "F3":3}
MAP_TAX = {"A1":1, "A2":2, "A3":3}

def yes_to_1(x):
    return 1 if isinstance(x, str) and x.strip().lower() == "yes" else 0

def encode_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Binary fields
    for c in ["GHG","Biodiversity","Water","Waste","Social"]:
        df[c] = df[c].apply(yes_to_1).astype(int)

    df["MIFID"] = df["MIFID"].apply(yes_to_1).astype(int)
    df["PAI_PREF"] = (df["PAI_PREF"].astype(str).str.lower() == "pai selected").astype(int)

    # Ordinals (conservative default to lowest tier)
    df["SI_CONSIDERATION_num"] = df["SI_CONSIDERATION_CD"].map(MAP_SI).fillna(1).astype(int)
    df["SFDR_PREF_num"] = df["SFDR_PREF"].map(MAP_SFDR).fillna(1).astype(int)
    df["SFDR_ACTUAL_num"] = df["SFDR_ACTUAL"].map(MAP_SFDR).fillna(1).astype(int)
    df["TAXONOMYPREF_num"] = df["TAXONOMYPREF"].map(MAP_TAX).fillna(1).astype(int)

    # SFDR engineered
    df["sfdr_gap"] = np.clip(df["SFDR_PREF_num"] - df["SFDR_ACTUAL_num"], -2, 2)
    df["sfdr_opp"] = np.maximum(df["sfdr_gap"], 0)  # 0..2

    # Topics aggregate
    topic_cols = ["GHG","Biodiversity","Water","Waste","Social"]
    df["esg_topics_yes_cnt"] = df[topic_cols].sum(axis=1)
    df["topics_norm"] = df["esg_topics_yes_cnt"] / len(topic_cols)

    # Normalized signals (0..1)
    df["si_norm"] = np.clip((df["SI_CONSIDERATION_num"] - 1)/2, 0, 1)
    df["sfdr_norm"] = np.clip(df["sfdr_opp"]/2, 0, 1)
    df["tax_norm"] = np.clip((df["TAXONOMYPREF_num"] - 1)/2, 0, 1)

    # PAI block (0..1)
    df["pai_block"] = np.where(df["PAI_PREF"]==1, 0.5 + 0.5*df["topics_norm"], 0.0)

    # Business logic helper
    df["si_is_s3"] = (df["SI_CONSIDERATION_num"] == 3).astype(int)

    df["si_offering"] = df["si_offering"].astype(int)
    return df

df_feat = encode_features(df_id)

# Sanity-check distributions
plt.figure(figsize=(7,4))
plt.hist(df_feat["sfdr_gap"], bins=5)
plt.title("Distribution: sfdr_gap (clipped)")
plt.xlabel("sfdr_gap")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.hist(df_feat["TAXONOMYPREF_num"], bins=3)
plt.title("Distribution: TAXONOMYPREF_num (A1/A2/A3 -> 1/2/3)")
plt.xlabel("TAXONOMYPREF_num")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

df_feat[["ID","si_offering","MIFID","SI_CONSIDERATION_num","si_is_s3","sfdr_gap","PAI_PREF","TAXONOMYPREF_num","esg_topics_yes_cnt"]].head()

---
## 5) Train/Validation split (ID-level)

All three methods are evaluated on the **same** validation set.

In [ ]:
FEATURES_BASE = ["MIFID","SI_CONSIDERATION_num","si_is_s3","sfdr_norm","pai_block","tax_norm"]

X = df_feat[FEATURES_BASE].copy()
y = df_feat["si_offering"].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
idx_train, idx_val = X_train.index, X_val.index

print("Train:", len(idx_train), "Val:", len(idx_val))
print("Train si_offering rate:", y_train.mean().round(4))
print("Val   si_offering rate:", y_val.mean().round(4))

---
## 6) Evaluation helpers (metrics + lift + calibration)

In [ ]:
def eval_scores(y_true, p, label):
    return {
        "model": label,
        "auc": roc_auc_score(y_true, p),
        "avg_precision": average_precision_score(y_true, p),
        "brier": brier_score_loss(y_true, p)
    }

def lift_table(y_true, p, n_bins=10):
    tmp = pd.DataFrame({"y": y_true, "p": p})
    tmp["bin"] = pd.qcut(tmp["p"], n_bins, labels=False, duplicates="drop") + 1
    return tmp.groupby("bin")["y"].agg(["mean","count"]).rename(columns={"mean":"si_rate"})

def plot_lift(tab, title):
    plt.figure(figsize=(8,4))
    plt.plot(tab.index, tab["si_rate"].values, marker="o")
    plt.title(title)
    plt.xlabel("Decile (1=lowest score, 10=highest)")
    plt.ylabel("si_offering rate")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_calibration(y_true, p, title):
    prob_true, prob_pred = calibration_curve(y_true, p, n_bins=10, strategy="quantile")
    plt.figure(figsize=(6,6))
    plt.plot(prob_pred, prob_true, marker="o")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Observed si_offering rate")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 7) Method 1 — Fixed rule (stakeholder weights)

### Logic implemented
- If `MIFID=0`: **only** SI_CONSIDERATION
- If `MIFID=1` and `SI=S3`: add confirmation from SFDR/PAI/Taxonomy
- If `MIFID=1` and `SI!=S3`: keep a conservative baseline

We also compute **“why columns”** (contributions) so each recommendation is explainable.

In [ ]:
@dataclass
class FixedRuleConfig:
    # MIFID=0 branch: only SI
    score_A_S1: float = 0
    score_A_S2: float = 45
    score_A_S3: float = 85

    # MIFID=1 branch:
    baseline_B_S1: float = 0
    baseline_B_S2: float = 30
    baseline_B_S3: float = 50

    confirm_max: float = 50  # max added if SI is high (S3)
    w_sfdr: float = 0.50
    w_pai: float = 0.30
    w_tax: float = 0.20

cfg_fixed = FixedRuleConfig()

def score_fixed_rule(df: pd.DataFrame, cfg: FixedRuleConfig) -> pd.DataFrame:
    df = df.copy()

    # Baselines
    base_A = df["SI_CONSIDERATION_num"].map({1: cfg.score_A_S1, 2: cfg.score_A_S2, 3: cfg.score_A_S3}).astype(float)
    base_B = df["SI_CONSIDERATION_num"].map({1: cfg.baseline_B_S1, 2: cfg.baseline_B_S2, 3: cfg.baseline_B_S3}).astype(float)

    confirm = cfg.confirm_max * (
        cfg.w_sfdr * df["sfdr_norm"] +
        cfg.w_pai * df["pai_block"] +
        cfg.w_tax * df["tax_norm"]
    )

    # Apply requested gate: only add confirm when MIFID=1 and SI=S3
    score = np.where(df["MIFID"]==0, base_A,
                     np.where(df["si_is_s3"]==1, base_B + confirm, base_B))

    df["score_fixed"] = np.clip(score, 0, 100)

    # Why columns
    df["why_base"] = np.where(df["MIFID"]==0, base_A, base_B)
    df["why_sfdr"] = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*cfg.w_sfdr*df["sfdr_norm"], 0.0)
    df["why_pai"]  = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*cfg.w_pai*df["pai_block"], 0.0)
    df["why_tax"]  = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*cfg.w_tax*df["tax_norm"], 0.0)

    return df

df_scored = score_fixed_rule(df_feat, cfg_fixed)

plt.figure(figsize=(7,4))
plt.hist(df_scored["score_fixed"], bins=30)
plt.title("Score distribution: Fixed rule")
plt.xlabel("score_fixed")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

df_scored[["score_fixed","why_base","why_sfdr","why_pai","why_tax"]].head()

In [ ]:
# Validation performance (Fixed rule)
p_fixed = df_scored.loc[idx_val, "score_fixed"].values / 100.0
fixed_metrics = eval_scores(y_val.values, p_fixed, "Fixed rule")
display(fixed_metrics)

lift_fixed = lift_table(y_val.values, p_fixed)
plot_lift(lift_fixed, "Lift (proxy): Fixed rule")
display(lift_fixed)

plot_calibration(y_val.values, p_fixed, "Calibration (proxy): Fixed rule")

---
## 8) Method 2 — Weighted rule (data-driven, still rule-like)

We keep the **same structure**, but learn the **confirmation weights** from data.

Training subset for learning weights:
- `MIFID=1` and `SI=S3` (only population where confirmation signals apply)

We fit logistic regression on train and convert positive coefficients into weights that sum to 1.

In [ ]:
train_df = df_scored.loc[idx_train].copy()
train_gate = train_df[(train_df["MIFID"]==1) & (train_df["si_is_s3"]==1)].copy()

WEIGHT_FEATURES = ["sfdr_norm","pai_block","tax_norm"]
if train_gate["si_offering"].nunique() < 2:
    print("Warning: gated training subset has only one class; falling back to fixed weights.")
    w_learned = pd.Series([cfg_fixed.w_sfdr, cfg_fixed.w_pai, cfg_fixed.w_tax], index=WEIGHT_FEATURES)
else:
    lr_w = LogisticRegression(max_iter=6000, class_weight="balanced")
    lr_w.fit(train_gate[WEIGHT_FEATURES], train_gate["si_offering"])
    coef = pd.Series(lr_w.coef_[0], index=WEIGHT_FEATURES)

    pos = np.maximum(coef.values, 0)  # stakeholder-friendly monotonic assumption
    if pos.sum() == 0:
        pos = np.ones_like(pos)
    w_learned = pd.Series(pos / pos.sum(), index=WEIGHT_FEATURES)

display(w_learned.to_frame("learned_weight"))

plt.figure(figsize=(7,4))
plt.bar(w_learned.index, w_learned.values)
plt.title("Weighted rule: learned confirmation weights (sum=1)")
plt.ylabel("weight")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
@dataclass
class WeightedRuleConfig:
    score_A_S1: float = cfg_fixed.score_A_S1
    score_A_S2: float = cfg_fixed.score_A_S2
    score_A_S3: float = cfg_fixed.score_A_S3

    baseline_B_S1: float = cfg_fixed.baseline_B_S1
    baseline_B_S2: float = cfg_fixed.baseline_B_S2
    baseline_B_S3: float = cfg_fixed.baseline_B_S3

    confirm_max: float = cfg_fixed.confirm_max

cfg_weighted = WeightedRuleConfig()

def score_weighted_rule(df: pd.DataFrame, cfg: WeightedRuleConfig, w: pd.Series) -> pd.DataFrame:
    df = df.copy()
    base_A = df["SI_CONSIDERATION_num"].map({1: cfg.score_A_S1, 2: cfg.score_A_S2, 3: cfg.score_A_S3}).astype(float)
    base_B = df["SI_CONSIDERATION_num"].map({1: cfg.baseline_B_S1, 2: cfg.baseline_B_S2, 3: cfg.baseline_B_S3}).astype(float)

    confirm = cfg.confirm_max * (
        w["sfdr_norm"] * df["sfdr_norm"] +
        w["pai_block"] * df["pai_block"] +
        w["tax_norm"] * df["tax_norm"]
    )

    score = np.where(df["MIFID"]==0, base_A,
                     np.where(df["si_is_s3"]==1, base_B + confirm, base_B))

    df["score_weighted"] = np.clip(score, 0, 100)

    df["whyW_base"] = np.where(df["MIFID"]==0, base_A, base_B)
    df["whyW_sfdr"] = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*w["sfdr_norm"]*df["sfdr_norm"], 0.0)
    df["whyW_pai"]  = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*w["pai_block"]*df["pai_block"], 0.0)
    df["whyW_tax"]  = np.where((df["MIFID"]==1) & (df["si_is_s3"]==1), cfg.confirm_max*w["tax_norm"]*df["tax_norm"], 0.0)
    return df

df_scored = score_weighted_rule(df_scored, cfg_weighted, w_learned)

plt.figure(figsize=(7,4))
plt.hist(df_scored["score_weighted"], bins=30)
plt.title("Score distribution: Weighted rule (data-driven)")
plt.xlabel("score_weighted")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

df_scored[["score_weighted","whyW_base","whyW_sfdr","whyW_pai","whyW_tax"]].head()

In [ ]:
# Validation performance (Weighted rule)
p_weighted = df_scored.loc[idx_val, "score_weighted"].values / 100.0
weighted_metrics = eval_scores(y_val.values, p_weighted, "Weighted rule (data-driven)")
display(weighted_metrics)

lift_weighted = lift_table(y_val.values, p_weighted)
plot_lift(lift_weighted, "Lift (proxy): Weighted rule")
display(lift_weighted)

plot_calibration(y_val.values, p_weighted, "Calibration (proxy): Weighted rule")

---
## 9) Method 3 — ML: Calibrated Logistic Regression (controlled enhancement)

We fit a calibrated LR to produce **probabilities**.

To respect the business structure, we include **interaction features** that approximate the gate:
- `gate = MIFID * si_is_s3`
- `gate * sfdr_norm`, `gate * pai_block`, `gate * tax_norm`

This allows ML to learn within the same conceptual framework.

In [ ]:
def make_ml_matrix(df: pd.DataFrame) -> pd.DataFrame:
    X = df[FEATURES_BASE].copy()
    X["gate"] = X["MIFID"] * X["si_is_s3"]
    X["gate_sfdr"] = X["gate"] * X["sfdr_norm"]
    X["gate_pai"]  = X["gate"] * X["pai_block"]
    X["gate_tax"]  = X["gate"] * X["tax_norm"]
    return X

Xtr = make_ml_matrix(df_scored.loc[idx_train])
Xva = make_ml_matrix(df_scored.loc[idx_val])

lr = LogisticRegression(max_iter=8000, class_weight="balanced")
cal_lr = CalibratedClassifierCV(lr, method="isotonic", cv=5)
cal_lr.fit(Xtr, y_train)

p_ml = cal_lr.predict_proba(Xva)[:,1]
ml_metrics = eval_scores(y_val.values, p_ml, "ML: Calibrated Logistic Regression")
display(ml_metrics)

lift_ml = lift_table(y_val.values, p_ml)
plot_lift(lift_ml, "Lift (proxy): ML Calibrated Logistic Regression")
display(lift_ml)

plot_calibration(y_val.values, p_ml, "Calibration (proxy): ML Calibrated Logistic Regression")

# Interpretability: coefficients from uncalibrated LR on same features
lr_plain = LogisticRegression(max_iter=8000, class_weight="balanced")
lr_plain.fit(Xtr, y_train)
coef = pd.Series(lr_plain.coef_[0], index=Xtr.columns).sort_values(key=np.abs, ascending=False)
display(coef.to_frame("coef"))

plt.figure(figsize=(10,4))
plt.bar(coef.index[:12], coef.values[:12])
plt.title("Top coefficients (uncalibrated LR; sign indicates direction)")
plt.ylabel("coef")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 10) Validation comparison (Fixed vs Weighted vs ML)

This is the stakeholder-ready head-to-head comparison on the **same** validation set.

In [ ]:
comparison = pd.DataFrame([fixed_metrics, weighted_metrics, ml_metrics]).round(4)
display(comparison.sort_values("auc", ascending=False))

for metric in ["auc","avg_precision","brier"]:
    plt.figure(figsize=(8,4))
    plt.bar(comparison["model"], comparison[metric])
    plt.title(f"Validation comparison: {metric}")
    plt.ylabel(metric)
    plt.xticks(rotation=30, ha="right")
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 11) Operational output: Top 20 IDs with `si_offering=0` + “why” columns

Operationally, use a score available for all IDs (Fixed or Weighted).

In [ ]:
RANK_METHOD = "score_weighted"  # "score_fixed" or "score_weighted"

df_out = df_scored.copy()
df_out["rank_score"] = df_out[RANK_METHOD]
df_out["score_percentile"] = (df_out["rank_score"].rank(pct=True) * 100).round(2)
df_out["bucket_3"] = pd.cut(df_out["score_percentile"], bins=[-0.01, 50, 80, 100], labels=["Low","Average","High"])

targets = df_out[df_out["si_offering"]==0].sort_values("rank_score", ascending=False)

if RANK_METHOD == "score_fixed":
    why_cols = ["why_base","why_sfdr","why_pai","why_tax"]
else:
    why_cols = ["whyW_base","whyW_sfdr","whyW_pai","whyW_tax"]

cols = ["ID","rank_score","score_percentile","bucket_3",
        "MIFID","SI_CONSIDERATION_num","sfdr_gap","PAI_PREF","TAXONOMYPREF_num","esg_topics_yes_cnt"] + why_cols

targets[cols].head(20)

---
## 12) Risks & governance (what stakeholders should know)

- **Proxy label bias**: `si_offering` is membership, not true interest.
- **Hard gate risk** (requested logic): if SFDR/PAI/Tax only apply when SI=S3, we may miss some opportunities.
- **Missing data**: defaults to lowest tier can under-score; monitor missingness.

Mitigations:
- Treat scores as prioritization; validate via pilot.
- Monitor lift and calibration over time.
- Refit weights quarterly (or after product/process changes).

---
## 13) Pilot plan (to get real labels)

Use this model to run a controlled pilot and collect **true outcomes**:
- Treatment: High bucket among `si_offering=0`
- Control: randomized sample from eligible pool
- Outcomes: response, meeting booked, adoption, pipeline created

Then retrain ML on true outcomes and re-compare Fixed vs Weighted vs ML.